# Data Init

In [2]:
import torch
import time
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from data.swarmset import ContinuingDataset, SwarmDataset
from networks.embedding import NoveltyEmbedding
from networks.archive import DataAggregationArchive
from networks.ensemble import Ensemble
import numpy as np
from scipy import ndimage
import random
from sklearn.manifold import TSNE
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [3]:
def getAttributeVector(controller):
    v0_l, v0_r = controller[0],controller[1]
    v1_l, v1_r = controller[2], controller[3]
    v0_l, v0_r, v1_l, v1_r = round(v0_l, 1), round(v0_r, 1), round(v1_l, 1), round(v1_r, 1)

    k = 0.5
    max_elem_score = max(-min(controllers[i]), max(controllers[i]))
    # max_elem_score = -max_elem_score if max_elem_score < k else max_elem_score

    k_2 = 0.75
    magnitude_score = np.linalg.norm(controllers[i])
    # magnitude_score = -magnitude_score if magnitude_score < k_2 else magnitude_score

    k_3 = 0.3
    average_score = np.average(np.sqrt(np.power(controllers[i], 2)))
    # average_score = -average_score if average_score < k_3 else average_score

    # Sensor off magnitude (trial i)
    on_magnitude = (v0_l**2) + (v0_r**2)

    # Sensor on magnitude (trial i)
    off_magnitude = (v1_l**2) + (v1_r**2)

    # Spinning Detection (sensor off - trial ii)
    if v0_l == 0.0 and v0_r == 0.0:
        off_spin_variance = 0.0
    else:
        denom = v0_l if v0_l != 0.0 else v0_r
        off_spin_variance = min(abs((v0_l + v0_r) / denom), 1.0)

    # Spinning Detection (sensor on - trial ii)
    if v1_l == 0.0 and v1_r == 0.0:
        on_spin_variance = 0.0
    else:
        denom = v1_l if v1_l != 0.0 else v1_r
        on_spin_variance = min(abs((v1_l + v1_r) / denom), 1)

    # Mirror Property
    mirrored_controller = np.array([v0_l, v0_r, -v0_l, -v0_r])
    mirror_score = np.linalg.norm(mirrored_controller - controllers[i])

    # Independence Property
    independent_controller = np.array([v0_l, v0_r, v0_l, v0_r])
    indep = np.linalg.norm(independent_controller - controllers[i])

    return [indep, mirror_score, on_spin_variance, off_spin_variance, on_magnitude, off_magnitude, max_elem_score, magnitude_score, average_score]

In [6]:
def CSVLineToVec(line):
    line_list = line.strip().replace("\n", "").split(",")
    float_list = []
    for i in line_list:
        float_list.append(float(i))
    float_list = np.array(float_list)
    return float_list

sampled_dataset = SwarmDataset("../data/full-mini", rank=0)
data = sampled_dataset

lim = 1000
controllers = []
classes = [-1 for i in range(lim)]
for i in range(lim):
    image, genome, behavior = sampled_dataset[i][0], sampled_dataset[i][1], sampled_dataset[i][2]
    for j in range(len(behavior)):
        if behavior[j] < 0.0:
            behavior[j] *= -1
    controllers.append(genome)

OUT = "../data/oracle"
with open(os.path.join(OUT, "original-hand-labeled-classes.txt"), "r") as f:
    lines = f.readlines()
    for line in lines:
        triplet = CSVLineToVec(line)
        classes[int(triplet[0])] = int(triplet[1])

training_data = []
testing_data = []
for i, controller in enumerate(controllers):
    if i % 10 == 0:
        testing_data.append([controller, getAttributeVector(controller), int(classes[i] > 0)])
    else:
        training_data.append([controller, getAttributeVector(controller), int(classes[i] > 0)])

print(len(training_data), len(testing_data))

900 100


# Network

In [14]:
import torch
import random
from torch import nn

device = "cuda" if torch.cuda.is_available() else "cpu"

class RandomDetectionNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.single_layer = nn.Sequential(
            nn.Linear(9, 1),
        )

    def forward(self, x):
        logits = self.single_layer(x)
        return logits

model = RandomDetectionNetwork().to(device)

In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
epochs = 10
for e in range(epochs):
    # Training
    for i, (controller, attributes, y_truth) in enumerate(training_data):
        X = torch.tensor(attributes).to(device)
        y = torch.tensor(y_truth).to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            loss, current = loss.item(), i
            print(f"loss: {loss:>7f}  [{current:>5d}/{len(training_data):>5d}]")

    # Testing
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for i, (controller, attributes, y_truth) in enumerate(testing_data):
            X = torch.tensor(attributes).to(device)
            y = torch.tensor(y_truth).to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (torch.round(pred) == y_truth)
    test_loss /= len(testing_data)
    correct /= len(testing_data)
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.